In [ ]:
import json
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr

In [ ]:
corona_df = pd.read_csv("../Data/processed/de_corona.csv")

weather_df = pd.read_csv("../Data/processed/de_weather.csv")

df = pd.read_csv("../Data/processed/de_corona_weather.csv")

df

In [ ]:
owid = pd.read_csv("../Data/external/Our_World_in_Data_DE_Covid-data.csv")
owid = owid[owid["iso_code"] == "DEU"][['date', 'stringency_index']].dropna()

df = df.merge(owid, on = "date")
df

In [ ]:
hdi = pd.read_csv("../Data/external/Subnational_HDI2.csv")

df = df.merge(hdi, on = "region_code")
df

In [ ]:
Xs = ["TemperatureAboveGround", "UVIndex"]
df = sm.add_constant(df)
Xs.append("const")

for region in set(df["iso3166-2"]):
    if region != "DK-81":
        df[region] = (df["iso3166-2"] == region).astype(int)
        Xs.append(region)
        
Xs.append("stringency_index")
est = sm.OLS(np.log(df["cases_pc"] + 1), df[Xs], hasconst = True).fit(cov_type = "cluster", cov_kwds = {"groups": df["iso3166-2"]}, use_t = True)
print(est.summary())

In [ ]:
Xs = ['TemperatureAboveGround', 'UVIndex']
Xs.append('const')
Xs.append('HDI')

est = sm.OLS(np.log(df["cases_pc"] + 1), df[Xs], hasconst = True).fit(cov_type = "cluster", cov_kwds = {"groups": df["iso3166-2"]}, use_t = True)
print(est.summary())